# Resurgence Case Study Notebook

This notebook demonstrates an end-to-end risk workflow:
1. Data ingestion for a sample multi-asset portfolio
2. Monte Carlo simulation for VaR/CVaR
3. Loss-distribution and rolling-VaR visualization
4. VaR/CVaR interpretation from a backtest summary
5. Deterministic auditor flagging with structured anomalies


In [ ]:
from __future__ import annotations

import asyncio
from datetime import date
from pathlib import Path

import numpy as np
import pandas as pd

from resurgence_py.auditor import LLMAuditor
from resurgence_py.engine import simulate_regime_var_cvar_python
from resurgence_py.models import CrashVolProfile, DataPullConfig, PortfolioSeries, RiskMetrics
from resurgence_py.visualization import plot_loss_distribution, plot_rolling_var
from validation.backtest import BacktestConfig, run_backtest_on_returns, write_backtest_results

np.set_printoptions(precision=6, suppress=True)
artifact_dir = Path('notebooks/artifacts')
artifact_dir.mkdir(parents=True, exist_ok=True)


## 1) Data Ingestion

The notebook tries to pull daily adjusted-close returns for `SPY/QQQ/TLT/GLD`.
If network data is unavailable, it falls back to a deterministic synthetic return series.


In [ ]:
def load_portfolio_returns() -> tuple[pd.Series, str]:
    try:
        import yfinance as yf

        tickers = ['SPY', 'QQQ', 'TLT', 'GLD']
        prices = yf.download(
            tickers=tickers,
            start='2018-01-01',
            end='2026-02-21',
            interval='1d',
            auto_adjust=False,
            progress=False,
            group_by='column',
            threads=True,
        )
        if prices.empty:
            raise RuntimeError('Empty yfinance payload')

        close = prices['Adj Close'] if 'Adj Close' in prices.columns.get_level_values(0) else prices['Close']
        returns = close.pct_change().dropna(how='any')
        weights = np.array([0.40, 0.25, 0.20, 0.15], dtype=np.float64)
        portfolio_returns = returns.mul(weights, axis=1).sum(axis=1)
        portfolio_returns.name = 'portfolio_return'
        source = 'yfinance'
    except Exception as exc:  # noqa: BLE001
        rng = np.random.default_rng(12)
        portfolio_returns = pd.Series(
            rng.normal(loc=0.00035, scale=0.011, size=1_000),
            index=pd.bdate_range(start='2020-01-02', periods=1_000),
            name='portfolio_return',
        )
        source = f'synthetic fallback ({exc.__class__.__name__})'

    return portfolio_returns.astype(np.float64), source


portfolio_returns, data_source = load_portfolio_returns()
print(f'data source: {data_source}')
print(f'observations: {len(portfolio_returns)}')
portfolio_returns.tail()


## 2) Simulation Run

Use the most recent 252 trading days as the calibration window and run a regime-switching Monte Carlo simulation.


In [ ]:
window_returns = portfolio_returns.iloc[-252:].to_numpy(dtype=np.float64)

risk, losses = simulate_regime_var_cvar_python(
    returns=window_returns.tolist(),
    confidence_level=0.95,
    simulations=50_000,
    horizon_days=10,
    transition_matrix=[[0.92, 0.04, 0.04], [0.10, 0.80, 0.10], [0.18, 0.14, 0.68]],
    state_vol_multipliers=[0.70, 1.85, 1.00],
    state_drift_adjustments=[0.0003, -0.0008, 0.0],
    initial_state=2,
    seed=42,
)

print(f"VaR(95%): {risk.var:.4%}")
print(f"CVaR(95%): {risk.cvar:.4%}")
print(f"Mean loss: {risk.mean_loss:.4%}")
print(f"Loss stddev: {risk.loss_stddev:.4%}")


## 3) Loss Distribution Visualization

Render the histogram from simulated losses and the rolling VaR/CVaR chart from backtest observations.


In [ ]:
try:
    loss_plot = plot_loss_distribution(
        losses=losses,
        output_path=artifact_dir / 'loss_distribution.png',
        bins=60,
        confidence_level=0.95,
    )
    print(f'loss distribution chart: {loss_plot}')
except RuntimeError as exc:  # matplotlib optional
    print(exc)


## 4) Rolling VaR/CVaR Backtest + Interpretation

Run a historical rolling backtest, write CSV/JSON outputs, and interpret breach behavior.


In [ ]:
backtest_config = BacktestConfig(
    tickers=['SPY', 'QQQ', 'TLT', 'GLD'],
    weights=[0.40, 0.25, 0.20, 0.15],
    confidence_level=0.95,
    rolling_window=126,
    method='historical',
    simulations=5_000,
    horizon_days=1,
    seed=42,
)

observations, summary = run_backtest_on_returns(
    portfolio_returns=portfolio_returns,
    config=backtest_config,
)

csv_path, json_path = write_backtest_results(
    observations=observations,
    summary=summary,
    output_dir=artifact_dir / 'validation',
)

print(f'backtest rows: {summary.observation_count}')
print(f'breaches: {summary.breaches} ({summary.breach_rate:.2%})')
print(f'expected breach rate: {summary.expected_breach_rate:.2%}')
print(f'Kupiec p-value: {summary.kupiec_pof.p_value:.4f}')
if summary.christoffersen.p_value is not None:
    print(f'Christoffersen p-value: {summary.christoffersen.p_value:.4f}')
print(f'csv output: {csv_path}')
print(f'json output: {json_path}')

try:
    rolling_plot = plot_rolling_var(observations, artifact_dir / 'rolling_var.png')
    print(f'rolling VaR chart: {rolling_plot}')
except RuntimeError as exc:  # matplotlib optional
    print(exc)

pd.DataFrame(
    {
        'metric': [
            'mean_predicted_var',
            'mean_predicted_cvar',
            'mean_realized_loss',
            'breach_rate',
            'kupiec_p_value',
        ],
        'value': [
            summary.mean_predicted_var,
            summary.mean_predicted_cvar,
            summary.mean_realized_loss,
            summary.breach_rate,
            summary.kupiec_pof.p_value,
        ],
    }
)


## 5) Auditor Flag Example

Create a stressed metrics payload to trigger deterministic anomaly checks and inspect the structured anomaly report.


In [ ]:
stressed_risk = RiskMetrics(
    var=max(risk.var * 0.70, 1e-6),
    cvar=max(risk.var * 0.50, 1e-6),  # intentionally below VaR to force a critical anomaly
    mean_loss=1.15,
    loss_stddev=1.10,
    confidence_level=risk.confidence_level,
    simulations=risk.simulations,
    horizon_days=risk.horizon_days,
    estimated_drift=risk.estimated_drift,
    estimated_volatility=2.5,
    sharpe_ratio=9.2,
    max_loss_zscore=13.5,
)

portfolio = PortfolioSeries(
    returns=portfolio_returns.tail(252).tolist(),
    annualized_volatility=float(np.std(portfolio_returns.tail(252), ddof=1) * np.sqrt(252.0)),
    observation_count=min(252, len(portfolio_returns)),
)

crash_profile = CrashVolProfile(
    volatility_2008=0.45,
    volatility_2020=0.55,
    sample_size_2008=252,
    sample_size_2020=252,
)

pull_config = DataPullConfig(
    start_date=date(2018, 1, 1),
    end_date=date(2026, 2, 21),
    request_timeout_s=20.0,
)

auditor = LLMAuditor(model_name='gpt-4o-mini', allow_live_llm=False)
audit_report = asyncio.run(
    auditor.run(
        risk=stressed_risk,
        portfolio=portfolio,
        crash_profile=crash_profile,
        losses=losses,
        pull_config=pull_config,
        rerun_count=0,
        max_reruns=2,
    )
)

print(f"severity: {audit_report.severity}")
print(f"flagged: {audit_report.flagged}")
print(f"requires rerun: {audit_report.requires_rerun}")
print(f"summary: {audit_report.summary}")

pd.DataFrame([anomaly.model_dump() for anomaly in audit_report.anomalies]).head(20)
